In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.special import kn
from scipy.integrate import quad
import csv

# NOTE FOR TOMORROW TO FIX THE FACT THAT T IS A STATIONARY NUMBER AND TAP INTO THE TEMP VALS

In [18]:
with open('Gstar.csv', 'r') as f:
    Gstarreader = csv.reader(f)
    Gstardata = {float(row[0]): float(row[1]) for row in Gstarreader}

with open('GstarS.csv', 'r') as f:
    GstarSreader = csv.reader(f)
    GstarSdata = {float(row[0]): float(row[1]) for row in GstarSreader}

In [19]:
# CONSTANTS
n = 1          
g = 2.0        # Degrees of freedom
c1 = np.sqrt(4*np.pi**3/45)
c2 = 2*np.pi**2/45
#T = 80

# COUPLING CONSTANTS
k_SV, k_SC = 1e-6, 1e-7 #Talk to Evan and change these numbers
Lambda = 1

# PARTICLE VELOCITIES
V_Pion = 1e3 #m/s If the pion is moving at ~1000m/s it has a similar cross-section to the rest
V_Kaon = 1e2
V_Baryon = 1e2

# Particle masses in MeV
m = 0.511      # Electron mass in MeV
M_Pl = 1.0  # Planck constant in MeV
M_Proton, M_Neutron, M_Lambda = 938.2720813, 939.57, 1115.683
M_ChargedPion, M_NeutralPion = 139.57039, 134.9768
M_ChargedKaon, M_NeutralKaon = 493.677, 497.611

In [20]:
def g_star(Temp):
    # Find closest temperature value in Gstardata
    closest_temp = min(Gstardata.keys(), key=lambda t: abs(t - Temp))
    return Gstardata[closest_temp]


def g_starS(Temp):
    # Find closest temperature value in GstarSdata
    closest_temp = min(GstarSdata.keys(), key=lambda t: abs(t - Temp))
    return GstarSdata[closest_temp]

In [21]:
def Y_EQ(x):
    return 0.145 * (g / g_starS(m / x)) * x**(3/2) * np.exp(-x) #eqn 5.25

# Define H(m) function (Hubble parameter)
def H_m(x):
    return (c1 * np.sqrt(g_star(m / x)) * (m / x)**2) / M_Pl

# Define s(x, m) function (entropy density)
def s(x, m):
    return c2 * g_star(m / x) * (m / x)**3

#change s to be the mandelstam S = (p1 + p2)^2 = (p3 + p4)^2
def MandelstamS(m1,m2,v1,v2):
    P_1 = 0.5*m1*(v1**2)
    P_2= 0.5*m2*(v2**2)
    return (P_1+P_2)**2

In [22]:
def Energy(Velocity, Mass): #E = sqrt(m^2 + p^2)
    P = 0.5*Mass*(Velocity**2)
    return np.sqrt(Mass**2 + P**2) #energy will change with the temperature. the cross section is (1/2s)(amplitude)
    
def sigma(s_minus_4m2, sigma_value):
    return sigma_value  # Replace this with your actual cross-section function

def thermally_averaged_cross_section(T, m, sigma_value):
    def integrand(s):
        return sigma(s - 4 * m**2, sigma_value) * np.sqrt(s) * kn(1, np.sqrt(s) / T)
    
    integral_result, error = quad(integrand, 4 * m**2, np.inf)
    return (1 / (8 * m**4 * T * kn(2, m / T))) * integral_result

def lambda_x(x, m, g_s, g_star, sigma_value):
    H_m_value = H_m(x)
    T = m/x
    thermally_avg_sigma = thermally_averaged_cross_section(T, m, sigma_value)
    
    return (x * thermally_avg_sigma * s_value) / H_m_value

def dYdx(x, Y, sigma_value):
    Y_eq = Y_EQ(x)
    lambda_val = lambda_x(x, m, g_starS, g_star, sigma_value)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)
    return np.array([result])

def get_variable_name(var, global_vars):
    names = [name for name, value in global_vars.items() if value is var]
    return names[0] if names else "unknown"

#### sigma = k^2/mp^2 where k is the coupling constant and mp is the mass of the mediating particle.
#### The strangeness conserving interactions and strangeness violating processes have different coupling constants.
#### k_violating > k_conserving
#### Take Mass of the particles to be equal to SM particles as of now.
#### sigma = sum_i^N sigma_i where N is the number of ways the particle can annihilate.
#### Need a Boltzmann equation for pi and K
#### Only look at processes that change total particle number
#### Strong coupling counstant is approx 1
#### Weak coupling constant is approx 10^-7
##### http://hyperphysics.phy-astr.gsu.edu/hbase/Forces/couple.html

#### ASSUMING THAT COUPLING CONSTANT OF STRANGENESS VIOLATING WEAK INTERACTIONS ARE 10^-6, STRANGENESS CONSERVING ARE 10^-7
### NAMING CONVENTION:
##### SV = Strangeness Violating, SC = Strangeness Conserving
##### Sigma_{Strangeness Conserving/Violating} _ {Particle Mediator} = (k_{Strangeness Conserving/Violating}/M_{Mediator Particle})**2

In [7]:
# PARTICLE ENERGIES
E_Pion = Energy(V_Pion, M_NeutralPion)
E_Kaon = Energy(V_Kaon, M_NeutralKaon)

# STRANGENESS VIOLATING CROSS-SECTIONS
Sigma_SV_Neutron = (k_SV/M_Neutron)**2
Sigma_SV_Proton = (k_SV/M_Proton)**2
Sigma_SV_Lambda = (k_SV/M_Lambda)**2
Sigma_SV_ChargedKaon = (k_SV/M_ChargedKaon)**2
Sigma_SV_ChargedPion = (k_SV/M_ChargedPion)**2
Sigma_SV_NeutralKaon = (k_SV/M_NeutralKaon)**2
Sigma_SV_NeutralPion = (k_SV/M_NeutralPion)**2

# STRANGENESS CONSERVING CROSS-SECTIONS
Sigma_SC_Neutron = (k_SC/M_Neutron)**2
Sigma_SC_Proton = (k_SC/M_Proton)**2
Sigma_SC_Lambda = (k_SC/M_Lambda)**2
Sigma_SC_ChargedKaon = (k_SC/M_ChargedKaon)**2
Sigma_SC_ChargedPion = (k_SC/M_ChargedPion)**2
Sigma_SC_NeutralKaon = (k_SC/M_NeutralKaon)**2
Sigma_SC_NeutralPion = (k_SC/M_NeutralPion)**2

# POINT CROSS-SECTION INTO DI-HIGGS
Sigma_Point_NeutralPion = ((Lambda**2)/(128*np.pi))*(1/E_Pion**2)
Sigma_Point_NeutralKaon = ((Lambda**2)/(128*np.pi))*(1/E_Kaon**2)

In [16]:
Y0 = [1e25]  
x_min = 3  
x_max = 10 
x_span = (x_min, x_max)
x_eval = np.linspace(x_min, x_max, 500000)

sigma_values = [Sigma_Point_NeutralKaon, Sigma_Point_NeutralPion]

for sigma_value in sigma_values:
    sigma_name = get_variable_name(sigma_value, globals())
    sol = solve_ivp(lambda x, Y: dYdx(x, Y, sigma_value), x_span, Y0, t_eval=x_eval)
        
    # Number of frozen-out dm particles
    Y_frozen = sol.y[0, -1]
    print(f'Frozen-out Y(x) for {sigma_name} = {sigma_value}: {Y_frozen}')
    
    plt.plot(sol.t, sol.y[0], label=f'{sigma_name} = {sigma_value}')

    
# Plot configurations
plt.yscale('log')  
#plt.xscale('log')  
plt.xlabel('x = m/T')
plt.ylabel('log(Y/Y(x=0))')
plt.title('Log plot of Y(x) for different $\sigma$ values')
plt.legend()
plt.grid(True)
#plt.savefig('DifferentSigmaSValues.png', facecolor='w')

plt.show()

NameError: name 's_value' is not defined

In [9]:
#Extra cell with dummy sigmas
"""
# Initial conditions and x range
Y0 = [1.0]  
x_min = 3  
x_max = 3e3 
x_span = (x_min, x_max)
x_eval = np.linspace(x_min, x_max, 50000)

# Values for sigma(s - 4m^2)
sigma_values = [1e1, 1e2, 1e3]

# Plotting the results for different sigma values
plt.figure(figsize=(8, 6))

for sigma_value in sigma_values:
    sol = solve_ivp(lambda x, Y: dYdx(x, Y, sigma_value), x_span, Y0, t_eval=x_eval)
    
    # Plot the current sigma_value
    plt.plot(sol.t, sol.y[0], label=f'sigma = {sigma_value}')

plt.yscale('log')  
plt.xscale('log')  
plt.xlabel('x = m/T')
plt.ylabel('log(Y/Y(x=0))')
plt.title('Log plot of Y(x) for different $\sigma$ values')
plt.legend()
plt.grid(True)
#plt.savefig('DifferentSigmaSValues.png', facecolor='w')

plt.show()
"""

"\n# Initial conditions and x range\nY0 = [1.0]  \nx_min = 3  \nx_max = 3e3 \nx_span = (x_min, x_max)\nx_eval = np.linspace(x_min, x_max, 50000)\n\n# Values for sigma(s - 4m^2)\nsigma_values = [1e1, 1e2, 1e3]\n\n# Plotting the results for different sigma values\nplt.figure(figsize=(8, 6))\n\nfor sigma_value in sigma_values:\n    sol = solve_ivp(lambda x, Y: dYdx(x, Y, sigma_value), x_span, Y0, t_eval=x_eval)\n    \n    # Plot the current sigma_value\n    plt.plot(sol.t, sol.y[0], label=f'sigma = {sigma_value}')\n\nplt.yscale('log')  \nplt.xscale('log')  \nplt.xlabel('x = m/T')\nplt.ylabel('log(Y/Y(x=0))')\nplt.title('Log plot of Y(x) for different $\\sigma$ values')\nplt.legend()\nplt.grid(True)\n#plt.savefig('DifferentSigmaSValues.png', facecolor='w')\n\nplt.show()\n"